In [ ]:
# CLASE 1 - GENERACIÓN DE DATOS SIMULADOS PARA E-COMMERCE

import numpy as np
import pandas as pd
import os

# Configuración del generador de números aleatorios
rng = np.random.default_rng(seed=42)
n_clientes = 500

# 1. GENERACIÓN DE DATOS BASE CON ARRAYS
ids = np.arange(1, n_clientes + 1)

nombres_base = np.array([
    "Ana", "Sofía", "Camila", "Valentina", "Isidora", "Martina",
    "Mateo", "Benjamín", "Tomás", "Vicente", "Lucas", "Joaquín",
    "Daniela", "Fernanda", "Ignacio", "Gabriel", "Antonia", "Catalina"
])

apellidos_base = np.array([
    "González", "Muñoz", "Rojas", "Díaz", "Pérez", "Soto",
    "Contreras", "Silva", "Martínez", "Sepúlveda", "Torres", "Flores"
])

# Creamos los nombres completos combinando aleatoriamente
nombres = rng.choice(nombres_base, size=n_clientes) + " " + rng.choice(apellidos_base, size=n_clientes)

# Ciudades con probabilidades específicas (Chile)
ciudades = np.array(["Santiago", "Valparaíso", "Concepción", "La Serena", "Antofagasta", "Temuco", "Rancagua"])
ciudad = rng.choice(ciudades, size=n_clientes, p=[0.45, 0.12, 0.14, 0.08, 0.08, 0.08, 0.05])

# 2. CÁLCULOS ESTADÍSTICOS VECTORIZADOS
# Edad: Distribución normal (promedio 35 años)
edad = np.clip(rng.normal(loc=35, scale=12, size=n_clientes), 18, 70).round(0).astype(int)

# Total_Compras: Distribución de Poisson (promedio 4.5 compras)
total_compras = rng.poisson(lam=4.5, size=n_clientes)
total_compras = np.clip(total_compras, 0, 35)

# Ticket promedio y Monto Total con ruido aleatorio
ticket_promedio = rng.lognormal(mean=np.log(25000), sigma=0.55, size=n_clientes)
monto_total = (total_compras * ticket_promedio) + rng.normal(0, 5000, size=n_clientes)
monto_total = np.clip(monto_total, 0, None).round(0).astype(int)

# 3. OPERACIONES MATEMÁTICAS BÁSICAS (NumPy)
print(f"--- Estadísticas Iniciales (NumPy) ---")
print(f"Monto Total de Ventas: ${np.sum(monto_total)}")
print(f"Promedio de Edad de Clientes: {np.mean(edad):.1f} años")
print(f"Máximo de compras por un cliente: {np.max(total_compras)}")

# 4. ESTRUCTURACIÓN Y GUARDADO
# Creamos la carpeta 'data' si no existe
if not os.path.exists('data'):
    os.makedirs('data')

# Creamos un DataFrame para visualización y exportación
clientes = pd.DataFrame({
    "ID": ids,
    "Nombre": nombres,
    "Edad": edad,
    "Ciudad": ciudad,
    "Total_Compras": total_compras,
    "Monto_Total": monto_total
})

# 5. GUARDAR Y PREPARAR PARA LA SIGUIENTE CLASE
# Guardamos en formato binario .npy para la siguiente clase
np.save(r'C:\Users\jceli\Bootcamp\proyecto-ecommerce-analytics\data\transacciones_iniciales.npy', monto_total)

# También exportamos el CSV inicial para la siguiente clase
clientes.to_csv(r'C:\Users\jceli\Bootcamp\proyecto-ecommerce-analytics\data\dataset_transacciones.csv', index=False)

print("\n✅ Proceso completado: Archivo 'data\transacciones_iniciales.npy' guardado.")
print("\n--- Primeros 5 registros del DataFrame ---")
print(clientes.head())

In [ ]:
# CLASE 2 - EXPLORACIÓN INICIAL DE DATOS CON PANDAS

import pandas as pd
import numpy as np

# 1. CARGA DE DATOS
# Cargamos el archivo que creamos con NumPy en clase 1
try:
    df = pd.read_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\dataset_transacciones.csv')
    data_numpy = np.load('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\transacciones_iniciales.npy')
    print("✅ Datos cargados exitosamente desde NumPy.\n")
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo .npy. Asegúrate de ejecutar la Clase 1 primero.")

# 2. EXPLORACIÓN INICIAL
print("\n--- Primeras 5 filas ---")
print(df.head())  # Visualizar primeras filas 

print("\n--- Últimas 5 filas ---")
print(df.tail())  # Visualizar últimas filas

print("\n--- Información General ---")
print(df.info())  # Inspección de tipos de datos y nulos 

print("\n--- Estadísticas Descriptivas ---")
print(df.describe())  # Estadísticas básicas 

# 3. FILTROS CONDICIONALES
# Ejemplo: Transacciones con monto total mayor a 100,000
ventas_altas = df[df['Monto_Total'] > 100000]
print(f"\n🚀 Cantidad de ventas > 100,000: {len(ventas_altas)}")

# Ejemplo: Clientes con más de 4 compras
clientes_frecuentes = df[df['Total_Compras'] > 4]
print(f"🛒 Cantidad de clientes frecuentes (>4 compras): {len(clientes_frecuentes)}")

# 4. SUMARIZACIÓN Y VALORES ÚNICOS
print("\n--- Clientes por Ciudad ---")
print(df['Ciudad'].value_counts())  # Conteo por categorías

print("\n--- Ciudades Únicas ---")
print(df['Ciudad'].unique())  # Identificar valores únicos

# 5. GUARDAR PARA LA SIGUIENTE CLASE (Limpieza)
df.to_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\dataset_explorado.csv', index=False)
print("\n✅ Dataset estructurado guardado como 'data/dataset_explorado.csv'.")

In [ ]:
# CLASE 3 - INTEGRACIÓN DE MÚLTIPLES FUENTES DE DATOS
import pandas as pd
import requests # Librería necesaria para consultar APIs

# 1. CARGA DEL CSV (Ventas del E-commerce)
    # Cargamos el archivo generado en la Clase 2
try:
    df_ventas = pd.read_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\dataset_explorado.csv')
    print("✅ CSV de ventas cargado exitosamente.")
except FileNotFoundError:
    print("⚠️ No se encontró el archivo. Verifique la Clase 2.")

# 2. CARGA DE EXCEL
try:
    # Simulamos un catálogo de categorías
    df_categorias = pd.DataFrame({
        'ID_Producto': [1, 2, 3, 4, 5],
        'Categoria': ['Electrónica', 'Hogar', 'Moda', 'Deportes', 'Juguetes']
    })
    df_categorias.to_excel('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\categorias_productos.xlsx', index=False)
    df_excel = pd.read_excel('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\categorias_productos.xlsx')
    print("✅ Archivo Excel de categorías cargado.")
except Exception as e:
    print(f"❌ Error con Excel: {e}")

# 3. EXTRACCIÓN WEB (API mindicador.cl)
try:
    url_api = "https://mindicador.cl/api/dolar"
    response = requests.get(url_api)
    data = response.json()
    
    # Extraemos el valor del dólar más reciente
    valor_dolar = data['serie'][0]['valor']
    print(f"✅ Valor del dólar extraído de la API: ${valor_dolar}")
except Exception as e:
    print(f"⚠️ Error al consultar la API: {e}")
    valor_dolar = 950.0         # Valor de respaldo (Plan de contingencia)

# Consulté a Gemini por errores como el 429, entre otros y me sugirió utilizar la página con API.
# Esto lo realicé con ayuda de Gemini, ya que no estaba en el material del curso.
# Es muy complejo extraer datos de páginas que tienen protecciones, incluso del sii.cl

# 4. UNIFICACIÓN DE FUENTES
# En esta etapa consolidamos la información para la limpieza futura
df_consolidado = df_ventas.copy()
df_consolidado['Valor_Dolar_Referencia'] = valor_dolar
df_consolidado['Fuente_Dolar'] = "mindicador.cl"

# 5. GUARDAR DATOS CONSOLIDADOS
# Consolidamos las ventas en un nuevo archivo para la Clase 4 (Limpieza)
df_consolidado.to_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\ventas_consolidadas.csv', index=False, sep=';')
print("\n Dataset consolidado guardado en 'data/ventas_consolidadas.csv' listo para limpieza.")

In [ ]:
# CLASE 4 - LIMPIEZA Y PREPARACIÓN DE DATOS PARA ANÁLISIS

import pandas as pd
import numpy as np

# 1. CARGA DE DATOS (Conexión con Clase 3)
try:
    df = pd.read_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\ventas_consolidadas.csv', sep=';', encoding='latin1')
    print("✅ Dataset cargado para limpieza.")
except FileNotFoundError:
    print("❌ Error: Ejecuta primero la Clase 3.")
    exit()

# Insertamos algunos nulos y duplicados aleatorios
df.loc[df.sample(frac=0.05).index, 'Edad'] = np.nan
df = pd.concat([df, df.iloc[:5]], ignore_index=True) 

# 2. GESTIÓN DE NULOS Y DUPLICADOS
print(f"\n Nulos detectados:\n{df.isnull().sum()}")
print(f" Duplicados iniciales: {df.duplicated().sum()}")

# Decisiones de limpieza:
df.drop_duplicates(inplace=True) # Eliminamos duplicados
df['Edad'] = df['Edad'].fillna(df['Edad'].median()) # Imputación por mediana

# 3. DETECCIÓN DE OUTLIERS (Método IQR)
# El IQR es ideal para variables financieras
Q1 = df['Monto_Total'].quantile(0.25)
Q3 = df['Monto_Total'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers = df[(df['Monto_Total'] < limite_inferior) | (df['Monto_Total'] > limite_superior)]
print(f"\n Outliers detectados en Monto_Total: {len(outliers)}")

# Filtramos el dataset para mantener solo datos "normales"
df_limpio = df[(df['Monto_Total'] >= limite_inferior) & (df['Monto_Total'] <= limite_superior)]

# 5. GUARDAR DATASET LIMPIO PARA LA SIGUIENTE CLASE
df_limpio.to_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\dataset_limpio.csv', index=False, sep=';', encoding='latin1')
print("\n Dataset limpio guardado en 'data/dataset_limpio.csv'.")

In [ ]:
# CLASE 5 - DATA WRANGLING Y TRANSFORMACIÓN AVANZADA

import pandas as pd
import numpy as np

# 1. CARGA DE DATOS (Conexión con Clase 4)
try:
    # Cargamos el dataset que ya no tiene nulos ni outliers extremos
    df = pd.read_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\dataset_limpio.csv', sep=';', encoding='latin1')
    print("✅ Dataset limpio cargado para Data Wrangling.")
except FileNotFoundError:
    print("❌ Error: Ejecuta primero la Clase 4.")
    exit()

# 2. TRANSFORMACIÓN DE TIPOS DE DATOS
# Aseguramos que los tipos sean correctos para cálculos y ahorro de memoria
df['ID'] = df['ID'].astype(int)
df['Total_Compras'] = df['Total_Compras'].astype(np.int32)
df['Monto_Total'] = df['Monto_Total'].astype(float)

# 3. CREACIÓN DE COLUMNAS CALCULADAS Y FUNCIONES LAMBDA
# Calculamos el Ticket Promedio por cliente
df['Ticket_Promedio'] = (df['Monto_Total'] / df['Total_Compras']).round(2)

# Aplicamos una función personalizada con lambda para categorizar por edad
df['Segmento_Etario'] = df['Edad'].apply(lambda x: 'Joven' if x < 30 else ('Adulto' if x < 60 else 'Sénior'))

# 4. NORMALIZACIÓN Y ESTRUCTURACIÓN DE DATOS
# Binning: Categorizamos el valor del cliente según su gasto
# Creamos 3 niveles: Bronce, Plata, Oro
bins = [0, 50000, 150000, np.inf]
labels = ['Bronce', 'Plata', 'Oro']
df['Rango_Valor_Cliente'] = pd.cut(df['Monto_Total'], bins=bins, labels=labels)

# Normalización simple: Score de compras (0 a 1) para identificar clientes activos
df['Score_Actividad'] = (df['Total_Compras'] - df['Total_Compras'].min()) / (df['Total_Compras'].max() - df['Total_Compras'].min())

# 5. ELIMINACIÓN DE DUPLICADOS RESIDUALES
# Por si la integración de fuentes generó duplicados nuevos
df = df.drop_duplicates()

# 6. EXPORTACIÓN FINAL PARA ANÁLISIS DE CLASE FINAL
df.to_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\dataset_final_wrangled.csv', index=False, sep=';', encoding='latin1')
print("\n Data Wrangling completado. Columnas nuevas: Ticket_Promedio, Segmento_Etario, Rango_Valor_Cliente.")
print(df[['Nombre', 'Segmento_Etario', 'Rango_Valor_Cliente', 'Score_Actividad']].head())

In [ ]:
# CLASE 6 - GENERACIÓN DE REPORTES Y TABLAS PIVOT

import pandas as pd
import numpy as np

# 1. CARGA DE DATOS 
# Conexión con Clases anteriores (1 a 5)
try:
    df = pd.read_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\dataset_final_wrangled.csv', sep=';', encoding='latin1')
    # Cargamos también el catálogo de Excel creado en la Clase 3
    df_cat = pd.read_excel('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\categorias_productos.xlsx')
    print("✅ Datasets cargados exitosamente.")
except Exception as e:
    print(f"❌ Error al cargar archivos. Vuelve a ejecutar clase 5 y 3: {e}")
    exit()

# 2. COMBINACIÓN DE DATOS (Merge)
# Generamos IDs de productos aleatorios para cruzar con el catálogo
df['ID_Producto'] = np.random.randint(1, 6, size=len(df))

# Unimos las tablas (Data Wrangling avanzado)
df_final = pd.merge(df, df_cat, on='ID_Producto', how='left')

# 3. AGRUPAMIENTO
# Obtenemos métricas resumidas por Ciudad y Categoría
resumen_ventas = df_final.groupby(['Ciudad', 'Categoria'])['Monto_Total'].agg(['sum', 'mean', 'count']).reset_index()
resumen_ventas.columns = ['Ciudad', 'Categoria', 'Venta_Total', 'Promedio_Venta', 'Num_Transacciones']

# 4. REESTRUCTURACIÓN (pivot_table)
# Creamos una tabla dinámica para ver el Total de Ventas por Categoría y Segmento (Binning)
# Creamos una matriz para comparar ventas de categorías entre ciudades
matriz_ventas = df_final.pivot_table(
    index='Ciudad', 
    columns='Categoria', 
    values='Monto_Total', 
    aggfunc='sum',
    fill_value=0
)

print("\n--- TABLA PIVOT: VENTAS POR CATEGORÍA Y SEGMENTO ---")
print(matriz_ventas)

# 5. EXPORTACIÓN FINAL DEL PROYECTO
# CSV para sistemas de datos y Excel para reportes gerenciales
df_final.to_csv('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\entrega_final_ventas.csv', index=False)
with pd.ExcelWriter('C:\\Users\\jceli\\Bootcamp\\proyecto-ecommerce-analytics\\data\\reporte_final_analisis.xlsx') as writer:
    df_final.to_excel(writer, sheet_name='Datos_Detallados', index=False)
    resumen_ventas.to_excel(writer, sheet_name='Resumen_KPIs', index=False)
    matriz_ventas.to_excel(writer, sheet_name='Matriz_Ciudades')

print("\n🏆 ¡Proyecto Finalizado! Archivos generados en la carpeta /data:")
print("- entrega_final_ventas.csv")
print("- reporte_final_analisis.xlsx (Con múltiples pestañas)")